In [1]:
from Acesso import Login
from Query import Query

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)


In [2]:
querys={
    
    'Devolução':
    
    """
    
    SELECT d.[Data de Entrada],d.Motivo,d.[Situação do Pedido],d.[Tipo de Operação],d.Pedido,d.NFe,
    d.[ID Cliente],c.[Nome Fantasia],COALESCE(c.Matriz,'AVULSO') AS [Matriz],s.Segmento,s.Canal,
    d.[ID Vendedor],v.[Nome Resumido],su.Equipe,su.Supervisor,
    d.SKU,p.Produto,p.Fabricante,p.Linha,
    d.Qtde,d.[Unid. VDA],d.[Qtde VDA],d.[Valor Unitário],
    d.[Total Geral]
    FROM netfeira.vw_devolucao d
    INNER JOIN netfeira.vw_cliente c ON d.[ID Cliente]=c.[ID Cliente]
    INNER JOIN netfeira.vw_segmento s ON c.[ID Segmento]=s.[ID Segmento]
    INNER JOIN netfeira.vw_vendedor v ON d.[ID Vendedor]=v.[ID Vendedor]
    INNER JOIN netfeira.vw_supervisor su ON v.[ID Equipe]=su.[ID Equipe]
    INNER JOIN netfeira.vw_produto p ON d.SKU=p.SKU
    WHERE [Tipo de Operação]='VENDAS' AND YEAR(d.[Data de Entrada])=YEAR(GETDATE())
    ORDER BY d.[Data de Entrada]    
    
    """,
    
    'Data':
    
    """
    
    DECLARE @DTBASE DATETIME, @DTFIM DATETIME, @DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=DATEADD(DAY,(DAY(@DTFIM)-1)*-1,@DTFIM)

    SELECT @DTINICIO AS [Data Mín],@DTFIM AS [Data Máx]    
    
    
    """,
    
    'Calendario':
    
    """
    
    SELECT * FROM netfeira.vw_calend
    
    """,
    
    'Vendas':
    
    """
    
    DECLARE @DTBASE DATETIME, @DTFIM DATETIME, @DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=DATEADD(DAY,(DAY(@DTFIM)-1)*-1,@DTFIM)

    SELECT SUM([Total Venda]) AS [Total Venda]
    FROM netfeira.vw_venda_estatico
    WHERE [Tipo de Operação]='VENDAS' AND [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM AND [Situação]='FATURADO'
    
    """
    
}

In [3]:
df=sql.CriarTabela(kwargs=querys)

In [4]:
df['Data']

,Data Mín,Data Máx
0,2023-03-01,2023-03-07


In [5]:
df['Devolução']=df['Devolução'].merge(df['Calendario'],left_on='Data de Entrada',right_on='Data',how='inner')

In [6]:
dt_min=df['Data'].loc[0,'Data Mín']

dt_max=df['Data'].loc[0,'Data Máx']

df['Consolidado']=df['Devolução'].loc[df['Devolução']['Data de Entrada'].between(dt_min,dt_max)]

In [12]:
df['Situacao']=df['Consolidado'].groupby(['Situação do Pedido'],as_index=False).agg({'Total Geral':'sum'})

In [23]:
total=df['Vendas']['Total Venda'].sum()

cancelado=df['Situacao'].loc[df['Situacao']['Situação do Pedido'].str.contains('CA'),'Total Geral'].sum()

devolucao=df['Situacao'].loc[df['Situacao']['Situação do Pedido'].str.contains('DEV'),'Total Geral'].sum()

perc_dev=round(devolucao/total,4)*100

perc_can=round(cancelado/total,4)*100

In [29]:
df['Motivo']=df['Consolidado'].loc[df['Consolidado']['Situação do Pedido'].str.contains('DEV')].groupby(['Motivo'],as_index=False).agg({'Total Geral':'sum'})

df['Motivo'].sort_values('Total Geral',ascending=False,ignore_index=True,inplace=True)

df['Motivo']


,Motivo,Total Geral
0,FALTA DE PRODUTO,14970.98
1,EM DESACORDO COM O PEDIDO,10572.31
2,VENDEDOR/CLIENTE SOLICITA CANC./ALTERAÇÃO,3839.00
3,ERRO DE DIGITAÇÃO,1485.51
4,PRODUTO AVARIADO,1396.30
5,PEDIDO DUPLICADO,1394.40
6,FORA DE TEMPERATURA,1092.82
7,MUDANÇA DE ENDEREÇO,901.56
8,PRODUTO PROXIMO AO VENCIMENTO,654.62
9,PRODUTO DUPLICADO,133.50


In [36]:
df['Mensal']=df['Devolução'].loc[df['Devolução']['Situação do Pedido'].str.contains('DEV')].groupby(['ID Mês','Mês'],as_index=False).agg({'Total Geral':'sum'})

df['Mensal'].sort_values('ID Mês',ascending=True,ignore_index=True,inplace=True)

df['Mensal']


,ID Mês,Mês,Total Geral
0,1,JANEIRO,122454.38
1,2,FEVEREIRO,167826.44
2,3,MARÇO,36441.00


In [35]:
df['Matriz']=df['Consolidado'].loc[df['Consolidado']['Situação do Pedido'].str.contains('DEV')].groupby(['Matriz'],as_index=False).agg({'Total Geral':'sum'})

df['Matriz'].sort_values('Total Geral',ascending=False,ignore_index=True,inplace=True)

df['Matriz']

,Matriz,Total Geral
0,AVULSO,17523.06
1,SONDA,7328.48
2,HOSP SAO LUIZ,3317.00
3,ST MARCHE,2737.76
4,REDE KRILL,2034.41
5,SACOLÃO SAÚDE,1914.82
6,ESPETO MADEIRA,564.76
7,LOPES SM,310.26
8,E-COMMERCE,298.26
9,REDE PLUS,115.14


In [38]:
df['Segmento']=df['Consolidado'].loc[df['Consolidado']['Situação do Pedido'].str.contains('DEV')].groupby(['Segmento'],as_index=False).agg({'Total Geral':'sum'})

df['Segmento'].sort_values('Total Geral',ascending=False,ignore_index=True,inplace=True)

df['Segmento']

,Segmento,Total Geral
0,10 OU + CHK,11340.17
1,RESTAURANTE,5012.19
2,HOSPITALIDADE,3727.86
3,SAÚDE,3317.00
4,5 A 9 CHK,3256.97
5,BAR/LANCHONETE,3155.89
6,SACOLÃO,1914.82
7,PADARIA/DELICATÉSSEN,1020.00
8,EMPÓRIO,970.32
9,AÇOUGUE,704.60
